In [1]:
import os
from groq import Groq
from dotenv import load_dotenv, find_dotenv
from retry import retry

_ = load_dotenv(find_dotenv())

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

@retry(tries=3, delay=5)
def get_completion(prompt: str, model: str) -> str:
    completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model,
        temperature=0,
    )
    return completion.choices[0].message.content

def get_reflexion_response(question: str, model: str) -> str:
    initial_response = get_completion(question, model)
    reflexion_prompt = f"""
    Considering the following question and initial response, try to critically reflect on the response, critique it.

    Question: {question}
    Initial response: {initial_response}
    Criticism:
    """
    reflexion = get_completion(reflexion_prompt, model)
    final_prompt = f""" 
    Considering the following question, initial response and criticism, try to come up with a final response.
    Question: {question}
    Initial response: {initial_response}
    Criticism: {reflexion}
    Final response:  

    """
    print("Prompt:", final_prompt)
    final_response = get_completion(final_prompt, model)
    return final_response

In [2]:
import pandas as pd 

df = pd.read_csv("crt_dataset.csv")
questions = df["Question"].tolist()

In [ ]:
model_to_responses = {}
MODELS = ["llama3-8b-8192", "llama3-70b-8192", "mixtral-8x7b-32768", "gemma-7b-it"]

for model in MODELS:
    model_to_responses[model] = []

    for question in questions:
        response = get_reflexion_response(question, model)
        model_to_responses[model].append(response)
        print(f"Question: {question}")
        print(f"Response: {response}")
        print("\n\n")

In [21]:
df = pd.DataFrame(model_to_responses)
df["Question"] = questions
df[["Question"] + MODELS].to_csv("agent_with_model_responses.csv")

In [27]:
print(get_reflexion_response(questions[30], "gemma-7b-it"))

Prompt:  
    Considering the following question, initial response and criticism, try to come up with a final response.
    Question: A woman purchases a chair for $100, sells it for $120, buys it back for $140, and finally sells it again for $160. How much profit did she make?
    Initial response: **Step 1: Calculate the profit from the first sale.**
- Selling price: $120
- Cost price: $100
- Profit: $120 - $100 = $20

**Step 2: Calculate the loss from the second sale.**
- Selling price: $140
- Cost price: $120
- Loss: $140 - $120 = $20

**Step 3: Calculate the profit from the final sale.**
- Selling price: $160
- Cost price: $140
- Profit: $160 - $140 = $20

**Total profit:**
- Profit from first sale: $20
- Loss from second sale: $20
- Profit from final sale: $20
- **Net profit:** $20 - $20 = **$0**

Therefore, the woman did not make any profit or loss on the chair.
    Criticism: ## Critique:

**Strengths:**

* The initial response provides a clear step-by-step explanation of the p

In [22]:
df

,llama3-8b-8192,llama3-70b-8192,mixtral-8x7b-32768,gemma-7b-it,Question
0,Here is a rewritten final response that addres...,Here is a revised final response that addresse...,"Taking into account the given question, initia...",## Final Response:\n\nThe provided question su...,Johnny’s mother had three children. The first ...
1,Here is a rewritten final response that addres...,Here's a revised final response that addresses...,Thank you for your thoughtful criticism. You'v...,## Final Response:\n\nThe owner of X's company...,The owner of X possesses four companies named ...
2,Here's a revised response that addresses the c...,Here's a revised final response that takes int...,Thank you for pointing out the nuance in the q...,## Final Response:\n\n**If you pass the runner...,"During a marathon, if you manage to pass the r..."
3,Here's a revised response that takes into acco...,Here's a revised final response that addresses...,If you outspell the contestant in third place ...,"**Final Response:**\n\nIn a spelling bee, if y...","In a spelling bee, if you outspell the contest..."
4,Here is a revised final response that addresse...,Here is a rewritten final response that incorp...,The initial response correctly highlights the ...,## Final Response:\n\nThe premise of the quest...,A train traveling from Berlin to Warsaw derail...
5,Here's a revised response that addresses the w...,Here's a rewritten final response that address...,The initial response correctly identifies that...,## Final Response:\n\nThe premise of the quest...,"On a stormy evening, a plane departs from Heat..."
6,Here's a revised response that addresses the w...,Here's a rewritten final response that address...,Thank you for the thoughtful criticism. You ra...,## Final Response:\n\nThe premise of the quest...,"A monkey, a squirrel, and a bird are competing..."
7,Here's a revised response that addresses the c...,Here's a revised final response that addresses...,Thank you for the thoughtful criticism. Here's...,## Final Response:\n\nThe premise of the quest...,"A monkey, a koala, and a parrot are competing ..."
8,"Based on the criticism, I would revise my init...",Here's a revised final response that addresses...,Thank you for the thoughtful criticism. You've...,## Final Response:\n\nThe passage suggests tha...,"In a blue bungalow, everything is blue: the wa..."
9,Here's a revised final response that addresses...,Here's a rewritten final response that incorpo...,The question leads us to focus on the color gr...,"## Final Response:\n\n""In a greenhouse where g...","In a one-storey green house, every detail is g..."
